# Qrylov

## Setup

In [ ]:
from typing import List, Tuple

import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import scipy as sp

import openfermion as of
from openfermionpyscf import run_pyscf

import numpy as np
import openfermion as of

import cirq
import qiskit
import qiskit.qasm2
import quimb.tensor as qtn

from tensor_network_common import pauli_sum_to_mpo

## Parameters

In [ ]:
# Problem to use.
problem: str = "product"  # Options: "hchain", "hubbard", "product", "reactant"

# For Hubbard Hamiltonian.
xdim: int = 2
ydim: int = 3

# For Hydrogen chains.
natoms: int = 10  # Number of H atoms in the (linear) chain.

hamiltonian_threshold: float = 0.02 # Drop terms in the Hamiltonian with coefficients smaller than this threshold.

# Initial state parameters.
seed: int = 1
alpha: float = 10.0  # The initial state is |ground state> + |random state> / alpha.

# Krylov parameters.
subspace_dimension: int = 16  # The dimension of the Krylov subspace.
threshold: float = -np.inf  # 1e-4  # -np.inf  # -1e-4  # Set to -np.inf to disable thresholding.

# Simulation parameters.
max_mpo_bond: int = 128  # Maximum bond dimension for MPO's.
max_mps_bond: int = 256  # Maximum bond dimension for MPS's.

In [ ]:
def fill_h_and_s_matrices(
    vectors: List[np.ndarray] | List[np.ndarray],
    mpo: qtn.MatrixProductOperator | np.ndarray,
    verbose: bool = False,
) -> Tuple[np.ndarray, np.ndarray]:
    tensor_algebra = isinstance(mpo, qtn.MatrixProductOperator)

    dim = len(vectors)
    h = np.zeros((dim, dim), dtype=np.complex128)
    s = np.zeros((dim, dim), dtype=np.complex128)

    for i in range(dim):
        for j in range(i, dim):
            if verbose:
                print(f"A_{i}{j}", end=" ")

            if tensor_algebra:
                hij = vectors[i].H @ mpo.apply(vectors[j])
            else:
                hij = vectors[i].conj().T @ mpo @ vectors[j]

            h[i, j] = hij
            if i != j:
                h[j, i] = np.conjugate(hij)

            if tensor_algebra:
                sij = vectors[i].H @ vectors[j]
            else:
                sij = vectors[i].conj().T @ vectors[j]
            s[i, j] = sij
            if i != j:
                s[j, i] = np.conjugate(sij)
    return h, s


# Based on https://quantum.cloud.ibm.com/docs/en/tutorials/krylov-quantum-diagonalization.
# and Algorithm 1.1 of https://arxiv.org/abs/2110.07492.
def solve_regularized_gen_eig(
    h: np.ndarray,
    s: np.ndarray,
    threshold: float = -np.inf,
) -> float:
    s_vals, s_vecs = sp.linalg.eigh(s)
    s_vecs = s_vecs.T
    good_vecs = np.array(
        [vec for val, vec in zip(s_vals, s_vecs) if val > threshold]
    )
    h_reg = good_vecs.conj() @ h @ good_vecs.T
    s_reg = good_vecs.conj() @ s @ good_vecs.T
    return sp.linalg.eigh(h_reg, s_reg)[0][0]

## Problem definition

In [ ]:
if problem == "hchain":
    bond_distance = 1.0
    geometry = [("H", (0, 0, i * bond_distance)) for i in range(natoms)]
    molecule = of.MolecularData(geometry, basis="sto-3g", multiplicity=1, charge=natoms % 2)

    molecule = run_pyscf(molecule, run_scf=True, run_mp2=True, verbose=True)

    hamiltonian = of.get_fermion_operator(molecule.get_molecular_hamiltonian())
    hamiltonian = of.jordan_wigner(hamiltonian)

elif problem == "hubbard":
    hamiltonian = of.jordan_wigner(
        of.hamiltonians.fermi_hubbard(
            xdim,
            ydim,
            1.0,
            1.0,
            spinless=True
        )
    )

elif problem in ("product", "reactant"):
    data = np.load(f"./hamiltonians/po3_1w_{problem}_22o.npz")
    ecore = data["e_nuc"]
    h1 = np.array(data["h1"])
    h2 = np.array(data["h2"])
    nelectrons = data["nelectron"]

    h1_new, h2_new = of.chem.molecular_data.spinorb_from_spatial(h1, h2)
    h = of.InteractionOperator(ecore.item(), h1_new, h2_new)
    hamiltonian = of.get_fermion_operator(h)
    if hamiltonian_threshold is not None:
        hamiltonian.compress(hamiltonian_threshold)
    hamiltonian = of.jordan_wigner(hamiltonian)

else:
    raise ValueError(f"Unknown problem {problem}.")

In [ ]:
if hamiltonian_threshold is not None:
    hamiltonian.compress(hamiltonian_threshold)

In [ ]:
nqubits = of.utils.count_qubits(hamiltonian)
nterms = len(hamiltonian.terms)
print(f"Hamiltonian acts on {nqubits} qubit(s) and has {nterms} Pauli terms.")

## Exact energy

In [ ]:
hamiltonian_cirq = of.qubit_operator_to_pauli_sum(hamiltonian)
mpo = pauli_sum_to_mpo(hamiltonian_cirq, cirq.LineQubit.range(nqubits), max_bond=max_mpo_bond, verbose=True)
mpo.show()

In [ ]:
dmrg = qtn.DMRG2(mpo, bond_dims=[2 ** (i + 1) for i in range(int(np.log2(max_mpo_bond)))])
dmrg.solve(verbosity=1)

energy_exact = np.real(dmrg.energy)
ground_state = dmrg.state
print("Exact energy:", energy_exact)
ground_state.show()

## Initial state

In [ ]:
bvec = ground_state + qtn.MPS_rand_state(mpo.L, bond_dim=2, seed=seed) / alpha
bvec /= bvec.norm()
overlap = np.abs(bvec @ ground_state) ** 2
overlap

## Krylov with $H$

In [ ]:
vectors = [bvec]
for i in range(subspace_dimension - 1):
    print(i)
    new = mpo.apply(vectors[-1], compress=True, max_bond=max_mps_bond)
    print("Max bond:", new.max_bond())
    vectors.append(new)

In [ ]:
h, s = fill_h_and_s_matrices(vectors, mpo, verbose=True)

In [ ]:
krylov_dvals = []
krylov_evals = []
for d in range(1, len(h) + 1):
    try:
        krylov_energy = solve_regularized_gen_eig(h[:d, :d], s[:d, :d], threshold=threshold)
        print(d, krylov_energy)
        krylov_dvals.append(d)
        krylov_evals.append(krylov_energy)
    except np.linalg.LinAlgError:
        continue

In [ ]:
plt.plot(krylov_dvals, krylov_evals, "--o", label="Krylov with $H$")
plt.axhline(energy_exact, ls="--", color="black", label="Exact")
plt.xlabel("Subspace dimension $d$")
plt.ylabel("Eigenvalue")
plt.legend();

### Unitary Krylov

In [ ]:
# See Theorem 3.1 of https://arxiv.org/abs/2110.07492.
# dt = np.pi / (evals_exact[-1] - evals_exact[0])
dt = np.pi / mpo.norm()

if problem == "hchain":
    dt *= 5 * natoms  # Empirically found that larger dt works better for H chains.

dt

In [ ]:
if nqubits <= 10:
    matrix = mpo.to_dense()
    vec = bvec.to_dense()

    vectors_u = []
    for k in range(-subspace_dimension // 2, subspace_dimension // 2 + 1, 1):
        print(k)
        Uk = sp.sparse.linalg.expm(-1j * matrix * k * dt)  # TODO: Speedup with https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.expm_multiply.html.
        vectors_u.append(Uk @ vec)


    h, s = fill_h_and_s_matrices(vectors_u, matrix, verbose=True)

    krylov_u_dvals = []
    krylov_u_evals = []
    for d in range(1, len(h)):
        try:
            krylov_energy = solve_regularized_gen_eig(h[:d, :d], s[:d, :d], threshold=threshold)
            print(d, krylov_energy)
            krylov_u_dvals.append(d)
            krylov_u_evals.append(krylov_energy)
        except np.linalg.LinAlgError:
            continue

## Trotterized unitary Krylov

In [ ]:
def cirq_pauli_sum_to_qiskit_pauli_op(pauli_sum: cirq.PauliSum) -> qiskit.quantum_info.SparsePauliOp:
    """Returns a qiskit.SparsePauliOp representation of the cirq.PauliSum."""
    cirq_pauli_to_str = {cirq.X: "X", cirq.Y: "Y", cirq.Z: "Z"}

    qubits = pauli_sum.qubits
    terms = []
    coeffs = []
    for term in pauli_sum:
        string = ""
        for qubit in qubits:
            if qubit not in term:
                string += "I"
            else:
                string += cirq_pauli_to_str[term[qubit]]
        terms.append(string)
        assert np.isclose(term.coefficient.imag, 0.0, atol=1e-7)
        coeffs.append(term.coefficient.real)
    return qiskit.quantum_info.SparsePauliOp(terms, coeffs)

In [ ]:
hamiltonian_qiskit = cirq_pauli_sum_to_qiskit_pauli_op(hamiltonian_cirq)

In [ ]:
ntrotter_values = [1, 2, 4, 8, 16, 32]
trotter_circuits = []
all_krylov_u_trotter_dvals = []
all_krylov_u_trotter_evals = []

for ntrotter in ntrotter_values:
    print("Status: ntrotter =", ntrotter)
    trotter_operation = qiskit.circuit.library.PauliEvolutionGate(
        hamiltonian_qiskit,
        time=dt,
        synthesis=qiskit.synthesis.LieTrotter(reps=ntrotter)
    )
    trotter_circuit = qiskit.QuantumCircuit(nqubits)
    trotter_circuit.append(trotter_operation, trotter_circuit.qubits)
    trotter_circuit = qiskit.transpile(
        trotter_circuit, basis_gates=["u3", "cx"]
    )  # TODO: Compile to a target backend, e.g. IBM Fez.
    print("Trotter circuit gate counts:")
    print(trotter_circuit.count_ops())

    trotter_circuits.append(trotter_circuit)

    forward = trotter_circuit.copy()
    forward_qasm = qiskit.qasm2.dumps(forward)

    reverse = trotter_circuit.inverse().copy()
    reverse_qasm = qiskit.qasm2.dumps(reverse)

    forward_vectors = [bvec]

    for k in range(subspace_dimension // 2):
        print("Forward:", k)
        new = qtn.CircuitMPS.from_openqasm2_str(
            forward_qasm, psi0=forward_vectors[-1], progbar=False, max_bond=max_mps_bond,
        ).psi
        forward_vectors.append(new)
    
    reverse_vectors = [bvec]
    for k in range(subspace_dimension // 2):
        print("Reverse:", k)
        new = qtn.CircuitMPS.from_openqasm2_str(
            reverse_qasm, psi0=reverse_vectors[-1], progbar=False, max_bond=max_mps_bond,
        ).psi
        reverse_vectors.append(new)

    _ = reverse_vectors.pop(0)
    reverse_vectors = list(reversed(reverse_vectors))

    vectors_u_trotter = reverse_vectors + forward_vectors

    h, s = fill_h_and_s_matrices(vectors_u_trotter, mpo, verbose=True)

    krylov_u_trotter_evals = []
    krylov_u_trotter_dvals = []
    for d in range(1, len(h)):
        try:
            krylov_energy = solve_regularized_gen_eig(h[:d, :d], s[:d, :d], threshold=threshold)
            print(f"Krylov {d} energy:", krylov_energy)
            krylov_u_trotter_dvals.append(d)
            krylov_u_trotter_evals.append(krylov_energy)
        except np.linalg.LinAlgError:
            continue
    
    all_krylov_u_trotter_dvals.append(krylov_u_trotter_dvals)
    all_krylov_u_trotter_evals.append(krylov_u_trotter_evals)
    print("\n" * 10)

In [ ]:
cutoff_ind = -1

kwargs = {"alpha": 0.6667, "mec": "black"}

# plt.figure(figsize=(8, 5))

errors_h = np.abs(np.array(krylov_evals) - energy_exact)
plt.plot(krylov_dvals[:cutoff_ind], errors_h[:cutoff_ind], "--s", label="$\mathcal{H}$", **kwargs)

if nqubits <= 10:
    errors_u = np.abs(np.array(krylov_u_evals) - energy_exact)
    plt.plot(krylov_u_dvals[:cutoff_ind], errors_u[:cutoff_ind], "--s", label="$U$", **kwargs)

for ntrotter, krylov_u_trotter_dvals, krylov_u_trotter_evals in zip(ntrotter_values, all_krylov_u_trotter_dvals, all_krylov_u_trotter_evals):
    errors_u_trotter = np.abs(np.array(krylov_u_trotter_evals) - energy_exact)
    plt.plot(krylov_u_trotter_dvals[:cutoff_ind], errors_u_trotter[:cutoff_ind], "--s", label=rf"$U'$ ({ntrotter} Trotter step(s))", **kwargs)

plt.axhline(1e-3, ls="--", color="black", label="Chemical accuracy")

# plt.title(f"{nqubits}-qubit Fermi-Hubbard Hamiltonian\nInitial state overlap with ground state = {round(overlap, 3)}")
plt.xlabel("Subspace dimension $d$")
plt.ylabel("Absolute energy error")

plt.yscale("log")
plt.legend(ncol=2) #bbox_to_anchor=(1.05, 1));
plt.tight_layout()
# plt.savefig(f"krylov_{problem}_nqubits_{nqubits}_overlap_{overlap}_threshold_{threshold}_max_trotter_{max(ntrotter_values)}_dt_{dt}_seed_{seed}.pdf")

In [ ]:
ngates_oneq = []
ngates_twoq = []
depths = []

for circuit in trotter_circuits:
    counts = circuit.count_ops()
    ngates_oneq.append(counts.get("u3"))
    ngates_twoq.append(counts.get("cx"))
    depths.append(circuit.depth())

In [ ]:
plt.plot(ntrotter_values, ngates_oneq, "--o", label="One-qubit gates")
plt.plot(ntrotter_values, ngates_twoq, "--o", label="Two-qubit gates")
plt.plot(ntrotter_values, depths, "--o", label="Circuit depth")

plt.xlabel("Number of Trotter steps")
plt.ylabel("Count")

plt.legend();